In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

column = 'claim_fulltext' # 'title', 'abstract' or 'claim_fulltext'

# Load Data for Core Cleantech Dictionary

In [2]:
# df_yake_cleantech_titles = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/cpc_yake_keywords_list_noun_chunks_embeddings.json')
# df_yake_cleantech_titles = pd.read_json("/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_015_co_occurrence_025_title.json")
df_yake_cleantech_titles = pd.read_json(f"/home/thiesen/Documents/Cleantech_Concepts/cleantech_keywords_similarity_005_co_occurrence_025_abstract.json")

# Load Data for Semantic Search enriched Co-Occurrence Analysis

In [2]:
df_yake_cleantech_similarity = pd.read_json('/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/Similarity Search/df_keyword_titles_cosine_similarity_radius_025_noun_chunks.json')

In [3]:
# Explode column keywords_bertforpatents
df_yake_cleantech_similarity = df_yake_cleantech_similarity.explode('keywords_bertforpatents')

In [4]:
# Build new dataframe out of all values in 'keyword_yake_lemma' and 'keywords_bertforpatents' column
combined_df = pd.concat([df_yake_cleantech_similarity['keyword_yake_lemma'], df_yake_cleantech_similarity['keywords_bertforpatents']]).drop_duplicates()
df_yake_cleantech_titles = pd.DataFrame(combined_df, columns=['keyword_yake_lemma'])

In [5]:
len(df_yake_cleantech_titles)

25304

# Load Data for Co-Occurrence Analysis

In [4]:
df_yake_uspto_epo_rel = pd.read_json(f'/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/df_keywords_list_agg_uspto_epo_rel_embeddings_{column}_noun_chunks.json')
# Drop columns keyword_yake_pos, keyword_yake_patentsberta_embedding, keyword_yake_climatebert_embedding and keyword_yake_bertforpatents_embedding
# df_yake_uspto_epo_rel.drop(columns=['keyword_yake_patentsberta_embedding', 'keyword_yake_climatebert_embedding', 'keyword_yake_bertforpatents_embedding'], inplace=True)

In [5]:
df_yake_cleantech_titles.head()

,keyword_yake_lemma,cleantech,keyword_yake_lemma_bertforpatents_embedding
0,aberrant,1,"[0.2693934143, 0.348465234, 0.6860293746, 1.31..."
1,abundant,1,"[0.7716676593, 0.1885169297, 0.1095398068, -0...."
2,accumulator,1,"[0.504627049, -0.3598301709, 0.3817014694, 0.2..."
3,accurate,1,"[-0.7603557110000001, 0.6554374695, -0.5015231..."
4,acidic,1,"[-0.2268049717, -0.6390919685, 0.3487114012, -..."


In [6]:
df_yake_uspto_epo_rel.head()

,keywords_yake_claim_fulltext_lemma,abs_frequency,patent_id,publn_nr,oaid,keywords_yake_claim_fulltext_patentsberta_embedding,keywords_yake_claim_fulltext_climatebert_embedding,keywords_yake_claim_fulltext_bertforpatents_embedding
0,aaa,35,"[5626983, 9949165, 6492046, 5191275, 8758921, ...","[1142043, 3363790, 3041827, 1042828, 2982004]",[],"[-0.009164794300000001, -0.5787734985, -0.3139...","[-0.0193257574, -0.0141516225, 0.0131431855, 0...","[-0.42265179750000004, -0.8338707089, -0.44803..."
1,aac,7,"[9067969, 7236059, 9524014, 6294352, 6417158, ...",[],[],"[-0.1441015601, -0.5350900292, -0.2306492031, ...","[0.025997683400000002, 0.0462871417, 0.0207896...","[0.48455637690000003, -0.2590337992, -0.608433..."
2,aar,5,"[5986108, 9458069]","[3103867, 2946009, 3385375]",[],"[-0.1567861736, -0.6682188511, -0.2236177772, ...","[-0.0596974939, 0.0217031986, 0.09825296700000...","[-0.3915314078, 0.0007248279, -0.1738779992, 0..."
3,aav,15,"[10119125, 9695220, 11104885, 10980896, 717289...","[3060575, 1135468, 3459965, 3062884]",[],"[-0.16056068240000002, -1.0209388733, -0.22905...","[-0.0716600269, -0.0178589281, 0.0750997365, -...","[0.0051265955, -0.9149596095, -0.2827564478000..."
4,aav capsid polypeptide,7,"[10119125, 9695220, 11104885, 10526584, 9719070]","[3060575, 3459965]",[],"[-0.1696463823, -0.8892566562, -0.203945100300...","[-0.0224281326, 0.1100657359, 0.04343590140000...","[-0.46955767270000004, 0.1143041328, -0.460525..."


In [7]:
# Extract unique keywords
cleantech_titles = df_yake_cleantech_titles['keyword_yake_lemma'].unique()
uspto_epo_rel_keywords = df_yake_uspto_epo_rel[f'keywords_yake_{column}_lemma'].unique()

# Initialize co-occurrence matrix
co_occurrence_matrix_ids = pd.DataFrame(0, index=cleantech_titles, columns=uspto_epo_rel_keywords)
co_occurrence_matrix_y02 = pd.DataFrame(0, index=cleantech_titles, columns=uspto_epo_rel_keywords)

# Co-Occurence by CPC Classification - yields too few results, threshold only leaves 9 unique keywords

In [ ]:
# Delete all spaces in list of strings in column 'cpc_class_symbol'
df_yake_uspto_epo_rel['cpc_class_symbol'] = df_yake_uspto_epo_rel['cpc_class_symbol'].progress_apply(lambda x: [i.replace(' ', '') for i in x])

In [ ]:
df_yake_uspto_epo_rel.sample(5)

In [ ]:
# Exploding the 'cpc_group' column while retaining the 'keyword_yake_lemma' column
cpc_group_exploded = df_yake_uspto_epo_rel.set_index('keyword_yake_lemma')['cpc_group'].explode().reset_index()
# Delete all rows where cpc_group is of type Float
cpc_group_exploded = cpc_group_exploded[cpc_group_exploded['cpc_group'].apply(lambda x: isinstance(x, str))]
# Aggregate on 'cpc_group', list of keyword_yake_lemma
cpc_group_exploded = cpc_group_exploded.groupby('cpc_group')['keyword_yake_lemma'].apply(list).reset_index()

In [ ]:
cpc_group_exploded.sample(5)

In [ ]:
# Exploding the 'cpc_class_symbol' column while retaining the 'keyword_yake' column
cpc_class_symbol_exploded = df_yake_uspto_epo_rel.set_index('keyword_yake_lemma')['cpc_class_symbol'].explode().reset_index()
# Delete all rows where cpc_group is of type Float
cpc_class_symbol_exploded = cpc_class_symbol_exploded[cpc_class_symbol_exploded['cpc_class_symbol'].apply(lambda x: isinstance(x, str))]
# Aggregate on 'cpc_group', list of keyword_yake_lemma
cpc_class_symbol_exploded = cpc_class_symbol_exploded.groupby('cpc_class_symbol')['keyword_yake_lemma'].apply(list).reset_index()

In [ ]:
cpc_class_symbol_exploded.sample(5)

In [ ]:
# Merge the two dataframes on 'cpc_group' and 'cpc_class_symbol', list of keyword_yake_lemma mind that keyword_yake_lemma is list of strings
cpc_group_class_symbol_merged = pd.merge(cpc_group_exploded, cpc_class_symbol_exploded, left_on='cpc_group', right_on='cpc_class_symbol', how='outer')
# New column cpc - if cpc_group is not NaN, then cpc = cpc_group, else cpc = cpc_class_symbol
cpc_group_class_symbol_merged['cpc'] = cpc_group_class_symbol_merged['cpc_group'].fillna(cpc_group_class_symbol_merged['cpc_class_symbol'])
# Merge keyword_yake_lemma_x and keyword_yake_lemma_y into one column keyword_yake_lemma
cpc_group_class_symbol_merged['keyword_yake_lemma'] = cpc_group_class_symbol_merged.progress_apply(
    lambda row: list(set(
        (row['keyword_yake_lemma_x'] if isinstance(row['keyword_yake_lemma_x'], list) else []) +
        (row['keyword_yake_lemma_y'] if isinstance(row['keyword_yake_lemma_y'], list) else [])
    )), axis=1)
# Delete columns cpc_group, cpc_class_symbol, keyword_yake_lemma_x and keyword_yake_lemma_y
cpc_group_class_symbol_merged.drop(columns=['cpc_group', 'cpc_class_symbol', 'keyword_yake_lemma_x', 'keyword_yake_lemma_y'], inplace=True)

In [ ]:
# Explode the 'keyword_yake_lemma' column
cpc_group_class_symbol_exploded = cpc_group_class_symbol_merged.explode('keyword_yake_lemma').reset_index(drop=True)

In [ ]:
# Merge cpc_group_class_symbol_exploded with df_yake_cleantech_titles on 'keyword_yake_lemma'and keep cpc column including duplicates
cpc_group_class_symbol_filtered = pd.merge(df_yake_cleantech_titles, cpc_group_class_symbol_exploded, how='inner', left_on='keyword_yake_lemma', right_on='keyword_yake_lemma', validate='many_to_many').reset_index(drop=True)

In [ ]:
# Delete all rows corresponding to 'keyword_yake_lemma' when keyword_yake_lemma occurs less than x times
co_occurrence_threshold = 100
co_occurrence = cpc_group_class_symbol_filtered['keyword_yake_lemma'].value_counts()
co_occurrence = co_occurrence[co_occurrence > co_occurrence_threshold]
co_occurrence = co_occurrence.reset_index()
co_occurrence.columns = ['keyword_yake_lemma', 'count']
cpc_group_class_symbol_filtered = cpc_group_class_symbol_filtered.merge(co_occurrence, how='inner', left_on='keyword_yake_lemma', right_on='keyword_yake_lemma', validate='many_to_many')

In [ ]:
cpc_group_class_symbol_filtered['keyword_yake_lemma'].nunique()

## Co-Occurence Matrix by CPC Classification

In [ ]:
# Function to update co-occurrence matrix
def update_co_occurrence(row):
    if isinstance(row['patent_id'], str) and row['patent_id'] in patent_id_dict:
        keyword_lists = patent_id_dict[row['patent_id']]
    elif isinstance(row['publn_nr'], str) and row['publn_nr'] in publn_nr_dict:
        keyword_lists = publn_nr_dict[row['publn_nr']]
    elif isinstance(row['oaid'], str) and row['oaid'] in oaid_dict:
        keyword_lists = oaid_dict[row['oaid']]
    else:
        return

    for keyword_list in keyword_lists:
        for keyword in keyword_list:
            co_occurrence_matrix_ids.at[row['keyword_yake_lemma'], keyword] += 1

# Apply the function to each row in the DataFrame
for index, row in tqdm(df_yake_uspto_epo_rel_titles_filtered.iterrows(), total=len(df_yake_uspto_epo_rel_titles_filtered)):
    update_co_occurrence(row)

# Co-Occurrence by ID

In [8]:
# Exploding the 'patent_id' column while retaining the 'keyword_yake' column
patent_id_exploded = df_yake_uspto_epo_rel.set_index(f'keywords_yake_{column}_lemma')['patent_id'].explode().reset_index()
# Put 'US' in front of each patent_id
patent_id_exploded['patent_id'] = 'US' + patent_id_exploded['patent_id'].astype(str)
# Delete all rows where patent_id contains the string None
patent_id_exploded = patent_id_exploded[~patent_id_exploded['patent_id'].str.contains('None')]
# Aggregate on 'patent_id', list of keyword_yake_lemma
patent_id_exploded = patent_id_exploded.groupby('patent_id')[f'keywords_yake_{column}_lemma'].apply(list).reset_index()

In [9]:
patent_id_exploded.head()

,patent_id,keywords_yake_claim_fulltext_lemma
0,US10000011,"[comprising forming, desired, support structure]"
1,US10000017,"[adhesion surface, attachment, mounting appara..."
2,US10000021,"[machining process, predetermined portion, sha..."
3,US10000025,[fiber orientation]
4,US10000033,"[blend, closure element, substantially flat]"


In [10]:
# Exploding the 'publn_nr' column while retaining the 'keyword_yake' column
publn_nr_exploded = df_yake_uspto_epo_rel.set_index(f'keywords_yake_{column}_lemma')['publn_nr'].explode().reset_index()
# Put 'EP' in front of each publn_nr
publn_nr_exploded['publn_nr'] = 'EP' + publn_nr_exploded['publn_nr'].astype(str)
# Delete all rows where publn_nr contains the string None
publn_nr_exploded = publn_nr_exploded[~publn_nr_exploded['publn_nr'].str.contains('None')]
# Aggregate on 'publn_nr', list all keyword_yake_lemma
publn_nr_exploded = publn_nr_exploded.groupby('publn_nr')[f'keywords_yake_{column}_lemma'].apply(list).reset_index()

In [11]:
publn_nr_exploded.head()

,publn_nr,keywords_yake_claim_fulltext_lemma
0,EP1000004,[organic phase]
1,EP1000028,"[acid addition salt, derivative, pharmaceutica..."
2,EP100005,[fleece]
3,EP100007,"[air andor oxygen, carrier particle]"
4,EP1000085,"[enriched, pure]"


In [12]:
# Exploding the 'oaid' column while retaining the 'keyword_yake' column
oaid_exploded = df_yake_uspto_epo_rel.set_index(f'keywords_yake_{column}_lemma')['oaid'].explode().reset_index()
# Put 'REL' in front of each oaid
oaid_exploded['oaid'] = 'REL' + oaid_exploded['oaid'].astype(str)
# Delete all rows where oaid contains the string None
oaid_exploded = oaid_exploded[~oaid_exploded['oaid'].str.contains('None')]
# Aggregate on 'oaid', list of 'keyword_yake_lemma'
oaid_exploded = oaid_exploded.groupby('oaid')[f'keywords_yake_{column}_lemma'].apply(list).reset_index()

In [13]:
oaid_exploded.head()

,oaid,keywords_yake_claim_fulltext_lemma
0,RELnan,"[aaa, aac, aar, aav, aav capsid polypeptide, a..."


In [14]:
# Concatenate the three exploded dataframes
# df_keywords_uspto_epo_rel = pd.concat([patent_id_exploded, publn_nr_exploded, oaid_exploded], axis=0).reset_index(drop=True)
df_keywords_uspto_epo_rel = pd.concat([patent_id_exploded, publn_nr_exploded], axis=0).reset_index(drop=True)

In [15]:
df_keywords_uspto_epo_rel.sample(5)

,patent_id,keywords_yake_claim_fulltext_lemma,publn_nr
228806,US5912919,"[early, late, local code sequence, symbol, tra...",NaN
156738,US4346151,[mouth],NaN
633194,NaN,"[controlled variable, waveguide]",EP3345027
234917,US6048643,[lithium intercalation compound],NaN
169273,US4579999,"[primary stage, recycle]",NaN


In [16]:
# Explode the 'keyword_yake_lemma' column
df_keywords_uspto_epo_rel_exploded = df_keywords_uspto_epo_rel.explode(f'keywords_yake_{column}_lemma').reset_index(drop=True)

In [17]:
len(df_keywords_uspto_epo_rel)

677010

In [18]:
# Merge df_keywords_uspto_epo_rel_exploded with df_yake_cleantech_titles on 'keyword_yake_lemma'and keep patent_id, publn_nr and oaid, including duplicates
df_yake_uspto_epo_rel_titles_filtered = pd.merge(df_yake_cleantech_titles, df_keywords_uspto_epo_rel_exploded, how='inner', left_on='keyword_yake_lemma', right_on=f'keywords_yake_{column}_lemma', validate='many_to_many')

In [19]:
# Delete all rows corresponding to 'keyword_yake_lemma' when keyword_yake_lemma occurs less than x times
co_occurrence_threshold = 100
co_occurrence = df_yake_uspto_epo_rel_titles_filtered[f'keywords_yake_{column}_lemma'].value_counts()
co_occurrence = co_occurrence[co_occurrence > co_occurrence_threshold]
co_occurrence = co_occurrence.reset_index()
co_occurrence.columns = ['keyword_yake_lemma', 'count']
df_yake_uspto_epo_rel_titles_filtered = df_yake_uspto_epo_rel_titles_filtered.merge(co_occurrence, how='inner', left_on='keyword_yake_lemma', right_on='keyword_yake_lemma', validate='many_to_many')

In [20]:
df_yake_uspto_epo_rel_titles_filtered[f'keywords_yake_{column}_lemma'].nunique()

297

## Co-Occurence Matrix by IDs

In [22]:
# Preprocess df_keywords_uspto_epo_rel to create dictionaries for fast lookup
patent_id_dict = df_keywords_uspto_epo_rel.groupby('patent_id')[f'keywords_yake_{column}_lemma'].progress_apply(list).to_dict()
publn_nr_dict = df_keywords_uspto_epo_rel.groupby('publn_nr')[f'keywords_yake_{column}_lemma'].progress_apply(list).to_dict()
# oaid_dict = df_keywords_uspto_epo_rel.groupby('oaid')[f'keywords_yake_{column}_lemma'].progress_apply(list).to_dict()

# Function to update co-occurrence matrix
def update_co_occurrence(row):
    if isinstance(row['patent_id'], str) and row['patent_id'] in patent_id_dict:
        keyword_lists = patent_id_dict[row['patent_id']]
    elif isinstance(row['publn_nr'], str) and row['publn_nr'] in publn_nr_dict:
        keyword_lists = publn_nr_dict[row['publn_nr']]
    elif isinstance(row['oaid'], str) and row['oaid'] in oaid_dict:
        keyword_lists = oaid_dict[row['oaid']]
    else:
        return

    for keyword_list in keyword_lists:
        for keyword in keyword_list:
            co_occurrence_matrix_ids.at[row['keyword_yake_lemma'], keyword] += 1

# Apply the function to each row in the DataFrame
for index, row in tqdm(df_yake_uspto_epo_rel_titles_filtered.iterrows(), total=len(df_yake_uspto_epo_rel_titles_filtered)):
    update_co_occurrence(row)

  6%|▌         | 28791/503825 [00:00<00:04, 97440.58it/s]

 91%|█████████ | 156991/173185 [00:03<00:00, 48685.01it/s] 


KeyboardInterrupt: 

In [45]:
# Print descending columns for row 'xxx'2
co_occurrence_matrix_ids.loc['wastewater treatment'].sort_values(ascending=False).head(20)

wastewater treatment    211
mfc                       6
microbial fuel cell       6
treatment system          5
anaerobic                 5
treatment plant           5
matter                    4
throughput                4
anammox                   4
algae                     3
constructed               3
bod                       3
biofilm                   3
environmental             3
effluent                  3
wwtp                      3
cod                       3
floc                      3
treatment process         3
wetland                   3
Name: wastewater treatment, dtype: int64

In [46]:
# Delete all rows with only zeros
co_occurrence_matrix_ids = co_occurrence_matrix_ids.loc[(co_occurrence_matrix_ids!=0).any(axis=1)]

In [47]:
# Divide rows in co-occurence matrix by largest value in row
co_occurrence_matrix_ids = co_occurrence_matrix_ids.div(co_occurrence_matrix_ids.max(axis=1), axis=0)

In [48]:
co_occurrence_matrix_ids.to_csv(f'/mnt/hdd01/patentsview/Similarity Search - CPC Classification and Claims/Co-Occurrence Analysis/co_occurrence_matrix_yake_keywords_cleantech_uspto_epo_rel_ids_semantic_similarity_{column}.csv', index=True)